In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import TruncatedSVD

In [11]:
rent = pd.read_csv("D:/project_data/KNN_data_rent.csv", encoding='euc-kr')
rent.head(2)

,Seoul,Gue,Name,Type,Open,Code,Add,Room,Howbig,Playground,Worker,Max,Now,When,X,Y,old_add,201809,predK25
0,서울특별시,송파,가락복지관어린이집,국공립,정상,5698,양재대로 897,4,220,1,12,70,57,1996-03-23,127.106891,37.495573,가락1동,37,0.902774
1,서울특별시,송파,개롱어린이집,국공립,정상,5820,동남로 225,3,74,1,8,28,28,2009-03-27,127.132103,37.498157,가락2동,1546,0.387448


In [12]:
rent_test = rent['predK25']
rent_test.head(3)

0    0.902774
1    0.387448
2    0.237042
Name: predK25, dtype: float64

In [17]:
rent_pred = rent[['predK25']]
rent_name = rent[['Name','predK25']]
rent_name.head(3)

,Name,predK25
0,가락복지관어린이집,0.902774
1,개롱어린이집,0.387448
2,은송어린이집,0.237042


In [18]:
all_data = pd.read_csv("D:/project_data/KK_k150_2021.csv", encoding='euc-kr')
all_data.head(3)

,Seoul,Gue,Name,Type,Open,Code,Add,Room,Howbig,Playground,...,Y,old_add,201809,predK25,k_cluster,center_access,people_access,202104,center_access_2,people_access_2
0,서울특별시,구로,연우어린이집,국공립,정상,8365,천왕로 50,4,313,1,...,37.481653,오류제2동,2449,0.275432,0,24.784041,0.422988,2412,23.986333,0.414978
1,서울특별시,구로,구립해사랑어린이집,국공립,정상,8366,광남로 54,3,103,0,...,37.475806,오류제2동,2449,0.280841,0,19.443473,0.422988,2412,20.796679,0.414978
2,서울특별시,구로,해뜨락어린이집,국공립,정상,8364,천왕로 9,4,224,0,...,37.478390,오류제2동,2449,0.275432,0,23.352479,0.422988,2412,33.044946,0.414978


In [19]:
all_data['rent'] = rent_pred
all_data.head(3)

,Seoul,Gue,Name,Type,Open,Code,Add,Room,Howbig,Playground,...,old_add,201809,predK25,k_cluster,center_access,people_access,202104,center_access_2,people_access_2,rent
0,서울특별시,구로,연우어린이집,국공립,정상,8365,천왕로 50,4,313,1,...,오류제2동,2449,0.275432,0,24.784041,0.422988,2412,23.986333,0.414978,0.902774
1,서울특별시,구로,구립해사랑어린이집,국공립,정상,8366,광남로 54,3,103,0,...,오류제2동,2449,0.280841,0,19.443473,0.422988,2412,20.796679,0.414978,0.387448
2,서울특별시,구로,해뜨락어린이집,국공립,정상,8364,천왕로 9,4,224,0,...,오류제2동,2449,0.275432,0,23.352479,0.422988,2412,33.044946,0.414978,0.237042


In [20]:
all_data[all_data['Name'] == rent_name.iloc[0,0]]

,Seoul,Gue,Name,Type,Open,Code,Add,Room,Howbig,Playground,...,old_add,201809,predK25,k_cluster,center_access,people_access,202104,center_access_2,people_access_2,rent
467,서울특별시,송파,가락복지관어린이집,국공립,정상,5698,양재대로 897,4,220,1,...,가락1동,37,0.60568,44,22.904301,0.438852,36,21.550478,0.396266,0.645349


In [21]:
all_data['rent'] = 0 # 더미 column
for i in range(len(rent_name)):
    all_data.iloc[all_data[all_data['Name'] == rent_name.iloc[i,0]].index[0], -1]  = rent_name.iloc[i, 1]  # index[0]467

In [22]:
all_data.head(3)

,Seoul,Gue,Name,Type,Open,Code,Add,Room,Howbig,Playground,...,old_add,201809,predK25,k_cluster,center_access,people_access,202104,center_access_2,people_access_2,rent
0,서울특별시,구로,연우어린이집,국공립,정상,8365,천왕로 50,4,313,1,...,오류제2동,2449,0.275432,0,24.784041,0.422988,2412,23.986333,0.414978,-0.167586
1,서울특별시,구로,구립해사랑어린이집,국공립,정상,8366,광남로 54,3,103,0,...,오류제2동,2449,0.280841,0,19.443473,0.422988,2412,20.796679,0.414978,-0.503908
2,서울특별시,구로,해뜨락어린이집,국공립,정상,8364,천왕로 9,4,224,0,...,오류제2동,2449,0.275432,0,23.352479,0.422988,2412,33.044946,0.414978,0.031126


In [23]:
X = all_data[['predK25', 'center_access', 'people_access','center_access_2', 'people_access_2', 'rent']]
X.head(3)
y = all_data[['Name']]

In [24]:
## scaling ##
ms = MinMaxScaler()
X_scale = ms.fit_transform(X)
#X_scale.(3)  #'numpy.ndarray' object has no attribute 'head'

X_scale[:3]
X_scale_inverse = ms.inverse_transform(X_scale)
X_scale_inverse[:3]

array([[ 2.75432462e-01,  2.47840406e+01,  4.22988026e-01,
         2.39863328e+01,  4.14977924e-01, -1.67586207e-01],
       [ 2.80841032e-01,  1.94434727e+01,  4.22988026e-01,
         2.07966786e+01,  4.14977924e-01, -5.03908046e-01],
       [ 2.75432462e-01,  2.33524794e+01,  4.22988026e-01,
         3.30449458e+01,  4.14977924e-01,  3.11264368e-02]])

In [25]:
## 전방행렬svd ##

svd = TruncatedSVD(n_components=3, random_state=77)
X_svd = svd.fit_transform(X_scale)
np.sum(svd.explained_variance_ratio_)   # 6 features -> 1 feature 로 축소할 때, 해당 1개의 feature가 나머지를 대표할 수 있는가? 신뢰도 문제

0.9467363557444067

In [26]:
X_svd = pd.DataFrame(X_svd)
X_svd.head(3)   # 0번째 col은 feature1개, 1번은 2개, 2번은 3개

,0,1,2
0,0.566079,0.166919,0.017015
1,0.497065,0.067263,0.075325
2,0.608927,0.226020,-0.018145


In [27]:
avg = np.mean(X_svd, axis = 1)
avg[:3]

0    0.250004
1    0.213218
2    0.272267
dtype: float64

In [28]:
X.shape # shape (1412, 6) 정사각형 만들기
X_sq = np.dot(X_scale.T, X_scale)
X_sq.shape  # (6, 6)

(6, 6)

In [29]:
U, S, V = np.linalg.svd(X_sq)
U.shape # (6, 6)
S.shape # (6,)
V.shape # (6, 6)
S

array([1164.26405981,   61.62741976,   17.25805933,    5.58211979,
          2.497306  ,    1.9140254 ])

In [30]:
U[:,0] #shape(6,1)
svd_test = np.dot(X_scale, U[:,:3]) #shape(1426, 6) (6,3) = (1426,3)

In [31]:
pd.DataFrame(svd_test).head(3)

,0,1,2
0,-0.566079,0.166919,-0.017015
1,-0.497065,0.067263,-0.075325
2,-0.608927,0.226020,0.018145


In [32]:
pd.DataFrame(X_svd).head(3)

,0,1,2
0,0.566079,0.166919,0.017015
1,0.497065,0.067263,0.075325
2,0.608927,0.226020,-0.018145


In [33]:
y['new'] = avg
y.head(3)

c:\users\403-17\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Name,new
0,연우어린이집,0.250004
1,구립해사랑어린이집,0.213218
2,해뜨락어린이집,0.272267


In [34]:
sorted_y = y.sort_values(['new'])
sorted_y.head(3)

,Name,new
833,도봉1동어린이집,0.068413
863,궁동어린이집,0.080753
865,항동어린이집,0.082644
